# Wasserstein distance estimation

In [1]:
import sys
sys.path.append("/home/lauro/code/msc-thesis/svgd")

import jax
import jax.numpy as np
from jax import grad, jit, vmap, random, lax, jacfwd
from jax import lax
from jax.ops import index_update, index
from jax.experimental import optimizers

import matplotlib.pyplot as plt
import numpy as onp
from tqdm import tqdm
import time
from functools import partial

import utils
import metrics
import plot
from svgd import SVGD
import svgd
import stein
import kernels
import train

rkey = random.PRNGKey(0)

from jax.config import config; config.update("jax_log_compiles", 1)

/home/lauro/.virtualenvs/msc-thesis/lib/python3.6/site-packages/jax/lib/xla_bridge.py:116: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


## Analytical Wasserstein distance between two gaussians

Let $\mu_{1}=\mathcal{N}\left(m_{1}, C_{1}\right)$ and $\mu_{2}=\mathcal{N}\left(m_{2}, C_{2}\right)$ be two non-degenerate Gaussian measures (i.e. normal distributions) on $\mathbb{R}^{n},$ with respective expected values $m_{1}$ and $m_{2} \in \mathbb{R}^{n}$ and symmetric positive semi-definite covariance matrices $C_{1}$ and $C_{2} \in \mathbb{R}^{n \times n} .$ Then, with respect to the usual Euclidean norm on $\mathbb{R}^{n},$ the 2
Wasserstein distance between $\mu_{1}$ and $\mu_{2}$ is

$$W_{2}\left(\mu_{1}, \mu_{2}\right)^{2}=\left\|m_{1}-m_{2}\right\|_{2}^{2}+\operatorname{trace}\left(C_{1}+C_{2}-2\left(C_{2}^{1 / 2} C_{1} C_{2}^{1 / 2}\right)^{1 / 2}\right)$$

## Wasserstein distance between two discrete distributions

See the scipy implementation [here](https://github.com/scipy/scipy/blob/adc4f4f7bab120ccfab9383aba272954a0a12fb0/scipy/stats/stats.py#L7094).

In [3]:
from stats import wasserstein_distance

In [4]:
a = [0, 1, 3]
b = [5, 6, 8]
wasserstein_distance(a, b)

/home/lauro/.virtualenvs/msc-thesis/lib/python3.6/site-packages/jax/lax/lax.py:5385: UserWarning: Explicitly requested dtype <class 'float'> requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/home/lauro/.virtualenvs/msc-thesis/lib/python3.6/site-packages/jax/lax/lax.py:5385: UserWarning: Explicitly requested dtype <class 'float'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))
/home/lauro/.virtualenvs/msc-thesis/lib/python3.6/site-packages/jax/numpy/lax_nump

DeviceArray(5., dtype=float32)

## Wasserstein distance between sample and density